In [1]:
from IPython.display import Markdown, display

display(Markdown("parcels.md"))

## [Property Tax Parcels](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)


MassGIS standardized assessors parcel mapping data set contains property (land lot) boundaries and database information from each community's assessor.   
Historic tax parcels are available.

Read the [overview](https://www.mass.gov/info-details/massgis-data-property-tax-parcels#overview-)


### Set-up

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv (
        find_dotenv (
            usecwd=True
        ),
    override=True
) # read local .env file and override any existing

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "ArlingtonMA")

#establish database connection for Transform queries and Loads
cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')


In [ ]:
import pandas as pd
import numpy  as np

### Extract

In [ ]:
def get_MassGIS_Level3_TaxParcels (
    url    = 'zip+http://download.massgis.digital.mass.gov/gdbs/l3parcels/M010_parcels_gdb.zip',
    debug  =  False
) :
    import  geopandas  as  gpd
    # for current use:
    # url = 'zip+http://download.massgis.digital.mass.gov/gdbs/l3parcels/M{dor}_parcels_gdb.zip'.format(dor=dor)

    ## Level3 Assessor Parcels with parcel geometry 
    ## in TaxPar layer and property information in Assess layer
    ## return EPSG 4326, lat/lon, coordinate system
   
    import fiona

    layers  =  fiona.listlayers ( url )
    TaxPar  =  layers [ 0 ]
    Assess  =  layers [ 1 ]
    l3      =  { }  
    
    for idx in range ( len ( layers ) ) :
        l3 [ layers [ idx ] ]  =  gpd . read_file ( url , layer = idx, dtype = str )
        if debug :
            print ( idx , layers [ idx ] , len ( l3 [ layers [ idx ] ] ) )
        
    ##layer 1 Assess has geomtery column all None
    l3 [ Assess ] . drop ( 'geometry' , axis = 1 , inplace = True )

    combo  =  l3 [ TaxPar ] . merge ( l3 [ Assess ] , on = [ 'TOWN_ID' , 'LOC_ID' ] , how = 'outer' , indicator = True )
    combo  =  combo [ combo . _merge == 'both' ] . reset_index ( drop = True )   ##63 no matches for Arlington; water, roads, etc.
    combo . drop ( '_merge' , axis = 1 , inplace = True )
    
    for col in [ 'YEAR_BUILT' , 'BLD_AREA' , 'UNITS' , 'RES_AREA' ] :
        combo [ col ] = combo [ col ] . fillna ( 0 ) . astype ( int )

    
    return  gpd . GeoDataFrame ( 
        combo , 
        geometry = combo . geometry
    ) . to_crs ( "EPSG:4326" )

### Transform

In [ ]:
def create_property_assessments_table(df):
    
    cols = ['FY','LOC_ID', 'PROP_ID', 'BLDG_VAL', 'LAND_VAL',
           'OTHER_VAL', 'TOTAL_VAL', 'LOT_SIZE', 'SHAPE_Area', 'LS_DATE', 'LS_PRICE',
           'USE_CODE', 'SITE_ADDR', 'ADDR_NUM', 'FULL_STR', 'LOCATION', 
           'ZIP', 'OWNER1', 'OWN_CITY', 'OWN_STATE',
           'LS_BOOK', 'LS_PAGE', 'ZONING', 'YEAR_BUILT',
           'BLD_AREA', 'UNITS', 'RES_AREA', 'STYLE', 'STORIES', 'NUM_ROOMS',
           'LOT_UNITS', 'geometry']
    newcols = ['year','loc_id','pid','building','land','other','total',
               'lot_size','area','last_sale_date','last_sale_price','land_use',
                'address','street_num','street_name','unit','zip',
               'owner','owner_city','owner_state','book','page',
               'zoning','year_built','building_area','units',
               'living_area','style','stories','rooms','lot_units','geometry']

    assess = df[cols].copy().rename(columns=dict(zip(cols,newcols)))

    assess['last_sale_date'] = pd.to_datetime(assess['last_sale_date']).dt.strftime('%Y-%m-%d')
    assess['area']   =  assess['area']/0.09290304 #sq meter to sq ft
    assess['unit']   =  assess['unit'].astype(str).str.replace("#","")
    assess['zoning'] =  assess['zoning'].str.upper()
    
    ## some early loc_id used meters instead of feet
    mask = assess.loc_id.str.contains('M_')
    if(mask.any()):
        print('M_ in loc_id')
    assess.loc[mask,'loc_id'] = 'F_'+\
        (assess.loc[mask,'loc_id'].str.split('_').str[1].astype(float)*3.280839895).apply(round,0).astype(int).astype(str) +\
        '_' +\
        (assess.loc[mask,'loc_id'].str.split('_').str[2].astype(float)*3.280839895).apply(round,0).astype(int).astype(str)
    
    ## enforce types
    for col in ['year','building','land_use','other','total','last_sale_price','area',
                'year_built','building_area','units','living_area','rooms']:
        assess[col]=assess[col].fillna(0).astype(int)

    assess = assess.drop(['lot_size','lot_units'],axis=1)

    return assess

def remove_dup_loc_id_round_geometry(df):
    import geopandas as gpd
    from shapely.wkt import loads
    import re

    simpledec = re.compile(r"\d*\.\d+")
    def mround(match):
        return "{:.7f}".format(float(match.group()))

    rdf = gpd.GeoDataFrame(df)
    rdf.geometry = rdf.geometry.apply(lambda x: loads(re.sub(simpledec, mround, x.wkt)))

    return rdf

def create_geometry(parcels):

    import geopandas as gpd
    
    parcels = parcels[parcels.year>2013].sort_values(['pid','year'])
    pid_2_address  =  parcels [ ['pid','street_name','street_num','unit'] ].copy()
    pid_2_address  =  pid_2_address . replace ( { None : '' , pd.isnull : '' } )
    pid_2_address  =  pid_2_address [ ~pid_2_address . duplicated ( ) ]\
                        . sort_values(['pid'])\
                        . reset_index(drop=True)
    
        
    loc_id_2_pid = parcels[['loc_id', 'pid']].sort_values(['loc_id', 'pid']).copy()
    loc_id_2_pid = loc_id_2_pid[ ~loc_id_2_pid.duplicated() ] \
                        . sort_values(['loc_id','pid'])\
                        . reset_index(drop=True)
    
    
    loc_polygons  =  parcels[['loc_id','year','geometry']].sort_values(['loc_id','year'])
    loc_polygons  =  loc_polygons[~loc_polygons.duplicated(['loc_id'])].sort_values(['loc_id','year']).reset_index(drop=True)
    
    loc_polygons [ 'lat'  ]  =  round ( loc_polygons . to_crs ( '+proj=cea' ) . geometry . centroid . to_crs ( loc_polygons . crs ) . y , 7 )
    loc_polygons [ 'lon'  ]  =  round ( loc_polygons . to_crs ( '+proj=cea' ) . geometry . centroid . to_crs ( loc_polygons . crs ) . x , 7 )
    
    loc_polygons = gpd.GeoDataFrame(loc_polygons,geometry=loc_polygons.geometry)



    ##221 PROP_IDs in 2013 but not in 2018
    ##issue of dup LOC_IDs within a few inches...
    mask = (parcels.year==2013)
    unique_from_2013 = parcels[mask][~parcels[mask].pid.isin(parcels[parcels.year>2013].pid.unique())][['loc_id','pid','geometry']]
    
    ##LOC_ID to pid xref
    mask = (parcels.year>2013)
    loc_pid = parcels[mask][['loc_id', 'year', 'pid']].sort_values(['loc_id', 'year', 'pid']).copy()
    loc_pid = loc_pid[ ~loc_pid.duplicated(['loc_id','pid']) ]
    loc_pid=loc_pid.sort_values(['loc_id','pid']).reset_index(drop=True)
    print('new loc_pid combos by year\n',loc_pid[loc_pid.year!=2018].groupby('year').count().to_markdown())
    
    return loc_pid, loc_polygons, pid_2_address


def normalize(parcels):
    
    query = """
                select ivp.key as ivp,substring(ivp.value,1,3)::int as value
                from common.int_value_pairs ivp
                where  ivp.item='land_use' order by ivp;
            """
    c2d_2_ivp={}
    for x,y in pd.read_sql(query,cnx).to_dict()['value'].items():
        c2d_2_ivp[y] = x
        
    parcels.land_use=parcels.land_use.replace(c2d_2_ivp)
    
    for col in ['zip', 'owner_city','owner_state',
                'zoning','style', 'stories'
               ]:
    
        query = """
                    select key, value
                    from common.int_value_pairs ivp
                    where  item='{col}';
                """.format(col=col)
        value_2_int={}
        for x,y in pd.read_sql(query,cnx).to_dict()['value'].items():
            value_2_int[y] = x
    
        parcels[col]=parcels[col].replace(value_2_int)
    
    ##periodically add new int_value_pairs
    int_value_pairs = pd.DataFrame()
    for col in ['owner_city','owner_state','zoning','stories']:
        tmp = sorted([x for x in parcels[col].unique() if type(x)==str])
        if len(tmp)>0:
            query = "select max(key) from common.int_value_pairs where item='{col}'"\
                        .format(col=col)
            N=pd.read_sql_query(query,cnx)['max'][0]
            
            ivp = pd.Series(tmp).reset_index().rename(columns={'index':'key',0:'value'})
            ivp['item']=col
            ivp['key']=N+1+ivp['key']
            int_value_pairs = pd.concat([int_value_pairs,ivp])
    
    print('int_value_pairs adds:\n',int_value_pairs.to_markdown())
    
    parcels=parcels.sort_values(['pid','year']).reset_index(drop=True)
    
    for col in ['land','total']:
        for period in [1,3,5]:
            parcels[col+'_chg_'+str(period)+'y']=\
                parcels.groupby('pid', sort=False, group_keys=False)[col]\
                    .apply(lambda x: x.pct_change(period))
            
        parcels[col+'_chg_10y']=\
            parcels.groupby('pid', sort=False, group_keys=False)[col]\
                .apply(lambda x: x.pct_change(7))
    
    
    for col in ['land','total']:
        for period in [1,3,5,10]:
            mask = ~pd.isnull(parcels[col+'_chg_'+str(period)+'y'])
            parcels.loc[mask,col+'_chg_'+str(period)+'y']=\
                parcels.loc[mask,col+'_chg_'+str(period)+'y'].round(6)
    
    
    parcels.stories=parcels.stories.astype(float).fillna(0).astype(int)
    
    parcels=parcels.replace({'^nan$':''},regex=True)

    return parcels,int_value_pairs

### Execute

In [ ]:
from glob import glob

data_dir = './parcels/'

data = {}
for file in sorted(glob ( data_dir + '*CY*zip' )) :
        key  =  file . split ( '_' ) [ -2 ] # FY

        data [ key ]  =  get_MassGIS_Level3_TaxParcels ( 'zip+file://'+file )
        print ( 'Working' , key )
        mask = data [ key ] . duplicated ( 'PROP_ID' )
        if mask . any ( ) :
            print ( len ( data [ key ] [ mask ] ) , 'Dups in PROP_ID' , key )

        ## ArlingtonMA changed street numbering 
        if key=='FY24':
            data[key].LOCATION=data[key].SITE_ADDR.str.split(" #").str[1]


parcels = pd.DataFrame()
for key in sorted(data.keys()):
    tmp = create_property_assessments_table(data[key])
    parcels=pd.concat([parcels,tmp])

## necessary?
parcels = remove_dup_loc_id_round_geometry(parcels)


print('parcel count\n',parcels[['year','pid']].groupby('year').count().to_markdown())

loc_pid, loc_polygons, pid_2_address = create_geometry(parcels)
parcels,int_value_pairs = normalize(parcels)

### Load

In [ ]:
loc_pid.to_sql(
        'loc_pid',
        schema='common',
        con=cnx,
        if_exists='append',
        index=False
    ) 

In [ ]:
parcels.drop(['geometry','address'],axis=1).to_sql(
        'parcels',
        schema='property',
        con=cnx,
        if_exists='append',
        index=False
    ) 

In [ ]:
loc_polygons.to_postgis(
    'loc_polygons',
    schema='common',
    con=cnx,
    if_exists='append',
    index=False
)

In [ ]:
int_value_pairs.to_sql(
        'int_value_pairs',
        schema='common',
        con=cnx,
        if_exists='append',
        index=False
    ) 
